In [32]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.17.0


# 二维卷积层

卷积神经网络（convolutional neural network）是含有卷积层（convolutional layer）的神经网络。本章中介绍的卷积神经网络均使用最常见的二维卷积层。它有高和宽两个空间维度，常用来处理图像数据。本节中，我们将介绍简单形式的二维卷积层的工作原理。

## 5.1.1 two dimentional cross-correlation

虽然卷积层得名于卷积（convolution）运算，但我们通常在卷积层中使用更加直观的互相关（cross-correlation）运算。在二维卷积层中，一个二维输入数组和一个二维核（kernel）数组通过互相关运算输出一个二维数组。
我们用一个具体例子来解释二维互相关运算的含义。如图5.1所示，输入是一个高和宽均为3的二维数组。我们将该数组的形状记为$3 \times 3$或（3，3）。核数组的高和宽分别为2。该数组在卷积计算中又称卷积核或过滤器（filter）。卷积核窗口（又称卷积窗口）的形状取决于卷积核的高和宽，即$2 \times 2$。图5.1中的阴影部分为第一个输出元素及其计算所使用的输入和核数组元素：$0\times0+1\times1+3\times2+4\times3=19$。

![二维互相关运算](../img/correlation.svg)

在二维互相关运算中，卷积窗口从输入数组的最左上方开始，按从左往右、从上往下的顺序，依次在输入数组上滑动。当卷积窗口滑动到某一位置时，窗口中的输入子数组与核数组按元素相乘并求和，得到输出数组中相应位置的元素。图5.1中的输出数组高和宽分别为2，其中的4个元素由二维互相关运算得出：

$$
0\times0+1\times1+3\times2+4\times3=19,\\
1\times0+2\times1+4\times2+5\times3=25,\\
3\times0+4\times1+6\times2+7\times3=37,\\
4\times0+5\times1+7\times2+8\times3=43.\\
$$

下面我们将上述过程实现在`corr2d`函数里。它接受输入数组`X`与核数组`K`，并输出数组`Y`。

1. **函数定义**：
```python
def corr2d(X, K):
```

- X：输入矩阵（可以是图像）
- K：卷积核（或称为滤波器）

2. **输出初始化**：
```python
h, w = K.shape
Y = tf.Variable(tf.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1)))
```

- 获取卷积核的高度h和宽度w
- 创建输出矩阵Y，尺寸为：
  - 高度 = X高度 - K高度 + 1
  - 宽度 = X宽度 - K宽度 + 1
- 初始化为全0矩阵

3. **卷积计算过程**：
```python
for i in range(Y.shape[0]):
    for j in range(Y.shape[1]):
        window_sum = tf.reduce_sum(X[i:i+h, j:j+w] * K)
        result = tf.cast(window_sum, dtype=tf.float32)
        Y[i,j].assign(result)
```

- 使用双重循环遍历输出矩阵的每个位置
- 对于每个位置：
  - 提取对应的输入窗口 X[i:i+h, j:j+w]
  - 与卷积核K进行元素乘法
  - 计算乘积的总和
  - 将结果转换为float32类型
  - 赋值给输出矩阵对应位置

这个函数的特点：
1. **手动实现**：
   - 没有使用TensorFlow的内置卷积操作
   - 清晰展示了卷积的计算过程

2. **教学价值**：
   - 帮助理解卷积运算的本质
   - 展示了滑动窗口的工作方式

3. **局限性**：
   - 计算效率较低（使用循环）
   - 没有考虑批量处理
   - 没有实现填充和步幅


In [33]:
def corr2d(X, K):
    h, w = K.shape
    Y = tf.Variable(tf.zeros((X.shape[0] - h + 1, X.shape[1] - w +1)))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            # 计算卷积窗口内的元素乘积和
            window_sum = tf.reduce_sum(X[i:i+h, j:j+w] * K)
            # 将结果转换为float32类型
            result = tf.cast(window_sum, dtype=tf.float32)
            # 将计算结果赋值给Y[i,j]
            Y[i,j].assign(result)
    return Y

我们可以构造图5.1中的输入数组`X`、核数组`K`来验证二维互相关运算的输出。

In [34]:
X = tf.constant([[0,1,2], [3,4,5], [6,7,8]])
K = tf.constant([[0,1], [2,3]])
corr2d(X, K)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[19., 25.],
       [37., 43.]], dtype=float32)>

## 5.1.2 Conv2d

二维卷积层将输入和卷积核做互相关运算，并加上一个标量偏差来得到输出。卷积层的模型参数包括了卷积核和标量偏差。在训练模型的时候，通常我们先对卷积核随机初始化，然后不断迭代卷积核和偏差。

下面基于`corr2d`函数来实现一个自定义的二维卷积层。在构造函数`__init__`里我们声明`weight`和`bias`这两个模型参数。前向计算函数`forward`则是直接调用`corr2d`函数再加上偏差。



















这段代码定义了一个自定义的二维卷积层类，用于理解卷积层的实现原理。让我详细解释其结构和功能：

1. **类定义**：
```python
class Conv2D(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units  # 定义卷积层的输出通道数
```


- 继承自tf.keras.layers.Layer
- 初始化时指定输出通道数
- 这是一个教学用的简化实现

2. **构建方法**：
```python
def build(self, kernel_size):
    self.w = self.add_weight(
        name='w',
        shape=kernel_size,
        initializer=tf.random_normal_initializer())
    
    self.b = self.add_weight(
        name='b',
        shape=(1,),
        initializer=tf.random_normal_initializer())
```


- 根据指定的kernel_size初始化卷积核
- 创建两个可训练参数：
  - w：卷积核权重，使用正态分布初始化
  - b：偏置项，同样使用正态分布初始化
  - 都是可训练的参数

3. **前向传播方法**：
```python
def call(self, inputs):
    return corr2d(inputs, self.w) + self.b
```


- 使用之前定义的corr2d函数执行卷积操作
- 加上偏置项
- 返回卷积结果

这个类的特点：
1. **教学目的**：
   - 简化的卷积层实现
   - 帮助理解卷积操作的基本原理

2. **局限性**：
   - 没有实现多通道输入
   - 没有实现多卷积核
   - 没有实现步幅和填充
   - 使用了效率较低的corr2d函数

3. **基本组成**：
   - 可训练的卷积核
   - 可训练的偏置项
   - 基本的卷积运算


In [35]:
# 该类在后续中没有被实际使用，仅仅是为了理解卷积层的实现原理
class Conv2D(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units  # 定义卷积层的输出通道数
    
    def build(self, kernel_size):
        # 初始化卷积核权重
        self.w = self.add_weight(
            name='w',
            shape=kernel_size,
            initializer=tf.random_normal_initializer())
        
        # 初始化偏置项
        self.b = self.add_weight(
            name='b',
            shape=(1,),
            initializer=tf.random_normal_initializer())
        
    def call(self, inputs):
        # 执行卷积操作并加上偏置项
        return corr2d(inputs, self.w) + self.b

卷积窗口形状为$p \times q$的卷积层称为$p \times q$卷积层。同样，$p \times q$卷积或$p \times q$卷积核说明卷积核的高和宽分别为$p$和$q$。

## 5.1.3 edge detection

下面我们来看一个卷积层的简单应用：检测图像中物体的边缘，即找到像素变化的位置。首先我们构造一张$6\times 8$的图像（即高和宽分别为6像素和8像素的图像）。它中间4列为黑（0），其余为白（1）。

In [36]:
X = tf.Variable(tf.ones((6,8)))
X[:, 2:6].assign(tf.zeros(X[:,2:6].shape))
X

<tf.Variable 'Variable:0' shape=(6, 8) dtype=float32, numpy=
array([[1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.]], dtype=float32)>

然后我们构造一个高和宽分别为1和2的卷积核`K`。当它与输入做互相关运算时，如果横向相邻元素相同，输出为0；否则输出为非0。

In [37]:
K = tf.constant([[1,-1]], dtype = tf.float32)

下面将输入`X`和我们设计的卷积核`K`做互相关运算。可以看出，我们将从白到黑的边缘和从黑到白的边缘分别检测成了1和-1。其余部分的输出全是0。

In [38]:
Y = corr2d(X, K)
Y

<tf.Variable 'Variable:0' shape=(6, 7) dtype=float32, numpy=
array([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.]], dtype=float32)>

由此，我们可以看出，卷积层可通过重复使用卷积核有效地表征局部空间。

## 5.1.4 learn kernel by data

最后我们来看一个例子，它使用物体边缘检测中的输入数据`X`和输出数据`Y`来学习我们构造的核数组`K`。我们首先构造一个卷积层，将其卷积核初始化成随机数组。接下来在每一次迭代中，我们使用平方误差来比较`Y`和卷积层的输出，然后计算梯度来更新权重。简单起见，这里的卷积层忽略了偏差。

虽然我们之前构造了`Conv2D`类，但由于`corr2d`使用了对单个元素赋值（`[i, j]=`）的操作因而无法自动求梯度。下面我们使用tf.keras.layers提供的`Conv2D`类来实现这个例子。

1. **输入数据重塑**：
```python
X = tf.reshape(X, (1,6,8,1))
Y = tf.reshape(Y, (1,6,7,1))
```


- 将2D数据转换为4D格式
- 维度含义：(批量大小, 高度, 宽度, 通道数)
- X从(6,8)变为(1,6,8,1)
- Y从(6,7)变为(1,6,7,1)
- 这是因为卷积层需要4D输入格式

2. **创建卷积层**：
```python
conv2d = tf.keras.layers.Conv2D(filters=1, kernel_size=(1, 2))
```


参数说明：
- filters=1：输出通道数为1，即只生成一个特征图
- kernel_size=(1,2)：使用1×2的卷积核
- 默认使用线性激活函数（无激活函数）

重要概念：
1. **维度要求**：
   - Conv2D层要求输入为4D张量
   - 格式：(batch_size, height, width, channels)

2. **卷积核参数**：
   - 形状维度：(kernel_height, kernel_width, input_channels, filters)
   - 这里是(1, 2, 1, 1)的卷积核

3. **数据预处理**：
   - 需要根据卷积层的要求调整输入数据维度
   - 添加批量维度和通道维度


In [72]:
# 将X的维度从(6, 8)调整为(1, 6, 8, 1)，以适应卷积层的输入要求
X = tf.reshape(X, (1,6,8,1))
# 将Y的维度从(6, 7)调整为(1, 6, 7, 1)，以适应卷积层的输出要求
Y = tf.reshape(Y, (1,6,7,1))
Y
        
# 创建一个2D卷积层
# filters=1: 定义输出通道数为1,即只产生一个特征图
# kernel_size=(1,2): 定义卷积核大小为1x2
# 具体API见Keras库
# GPT的Prompt链接：https://g.co/gemini/share/837a8138956a
# Conv2D形状维度: (kernel_height, kernel_width, input_channels, filters)
# 默认激活函数为linear
conv2d = tf.keras.layers.Conv2D(filters=1, kernel_size=(1, 2))





















这段代码实现了卷积层参数的训练过程。让我详细解释其工作流程：

1. **初始化**：
```python
Y_hat = conv2d(X)
```

- 使用卷积层对输入X进行初始预测

2. **训练循环**：
```python
for i in range(100):
```

- 设置最大迭代次数为100
- 使用梯度下降优化卷积层参数

3. **前向传播和损失计算**：
```python
with tf.GradientTape() as g:
    Y_hat = conv2d(X)
    l = tf.reduce_sum((Y_hat - Y) ** 2)
    if l < 1e-3:
        break
```

- 使用GradientTape记录计算过程
- 计算卷积层输出
- 计算均方误差损失
- 当损失小于阈值时提前结束训练

4. **梯度计算和参数更新**：
```python
dl = g.gradient(l, conv2d.trainable_variables)
lr = 3e-2
conv2d.trainable_variables[0].assign_sub(lr * dl[0])
```

- 计算损失对参数的梯度
- 设置学习率为0.03
- 仅更新部分权重参数（避免损失发散）

5. **训练监控**：
```python
if (i + 1) % 2 == 0:
    print('批次 %d, 损失 %.3f' % (i + 1, l))
```

- 每两次迭代打印一次损失值
- 监控训练进度

重要说明：
1. **参数更新策略**：
   - 只更新部分权重参数
   - 注释中说明更新所有参数会导致损失发散

2. **提前停止条件**：
   - 当损失小于1e-3时停止训练
   - 防止过度训练

3. **训练特点**：
   - 使用简单的梯度下降
   - 固定学习率
   - 手动实现的训练循环

这段代码展示了：
- 如何训练卷积层的参数
- 梯度下降的基本实现
- 训练过程中的注意事项
- 如何监控训练进度

In [73]:
# 初始化卷积层的输出
Y_hat = conv2d(X)

# 进行10次迭代训练
for i in range(100):
    # 使用GradientTape记录梯度
    with tf.GradientTape() as g:
        # 计算卷积层的输出
        Y_hat = conv2d(X)
        # 计算均方误差损失
        l = tf.reduce_sum((Y_hat - Y) ** 2)
        if l < 1e-3:
            break
        
    
    # 计算损失对卷积层参数的梯度
    dl = g.gradient(l, conv2d.trainable_variables)
    
    # 设置学习率
    lr = 3e-2
    
    # 反向传播，更新部分权重参数
    conv2d.trainable_variables[0].assign_sub(lr * dl[0])
    # 更新所有权重参数
    # 实验表示更新全部权重会导致Loss发散
    # 批次 2, 损失 58.790
    # 批次 4, 损失 3209.656
    # 批次 6, 损失 178068.516
    # 批次 8, 损失 9879409.000
    # 批次 10, 损失 548118784.000
    # for j, grad in enumerate(dl):
    #     conv2d.trainable_variables[j].assign_sub(lr * grad)
    
    # 每两次迭代打印一次损失
    if (i + 1) % 2 == 0:
        print('批次 %d, 损失 %.3f' % (i + 1, l))

批次 2, 损失 20.170
批次 4, 损失 3.611
批次 6, 损失 0.699
批次 8, 损失 0.155
批次 10, 损失 0.042
批次 12, 损失 0.013
批次 14, 损失 0.005
批次 16, 损失 0.002


可以看到，10次迭代后误差已经降到了一个比较小的值。现在来看一下学习到的核数组。

In [76]:
print("权重形状", conv2d.get_weights()[0].shape)
print("偏置形状", conv2d.get_weights()[1].shape)
print("激活函数名称:", conv2d.activation.__name__)


权重形状 (1, 2, 1, 1)
偏置形状 (1,)
激活函数名称: linear


可以看到，学到的核数组与我们之前定义的核数组`K`较接近。

## 互相关运算和卷积运算

实际上，卷积运算与互相关运算类似。为了得到卷积运算的输出，我们只需将核数组左右翻转并上下翻转，再与输入数组做互相关运算。可见，卷积运算和互相关运算虽然类似，但如果它们使用相同的核数组，对于同一个输入，输出往往并不相同。

那么，你也许会好奇卷积层为何能使用互相关运算替代卷积运算。其实，在深度学习中核数组都是学出来的：卷积层无论使用互相关运算或卷积运算都不影响模型预测时的输出。为了解释这一点，假设卷积层使用互相关运算学出图5.1中的核数组。设其他条件不变，使用卷积运算学出的核数组即图5.1中的核数组按上下、左右翻转。也就是说，图5.1中的输入与学出的已翻转的核数组再做卷积运算时，依然得到图5.1中的输出。为了与大多数深度学习文献一致，如无特别说明，本书中提到的卷积运算均指互相关运算。


## 特征图和感受野

二维卷积层输出的二维数组可以看作是输入在空间维度（宽和高）上某一级的表征，也叫特征图（feature map）。影响元素$x$的前向计算的所有可能输入区域（可能大于输入的实际尺寸）叫做$x$的感受野（receptive field）。以图5.1为例，输入中阴影部分的四个元素是输出中阴影部分元素的感受野。我们将图5.1中形状为$2 \times 2$的输出记为$Y$，并考虑一个更深的卷积神经网络：将$Y$与另一个形状为$2 \times 2$的核数组做互相关运算，输出单个元素$z$。那么，$z$在$Y$上的感受野包括$Y$的全部四个元素，在输入上的感受野包括其中全部9个元素。可见，我们可以通过更深的卷积神经网络使特征图中单个元素的感受野变得更加广阔，从而捕捉输入上更大尺寸的特征。

我们常使用“元素”一词来描述数组或矩阵中的成员。在神经网络的术语中，这些元素也可称为“单元”。当含义明确时，本书不对这两个术语做严格区分。


## 小结

* 二维卷积层的核心计算是二维互相关运算。在最简单的形式下，它对二维输入数据和卷积核做互相关运算然后加上偏差。
* 我们可以设计卷积核来检测图像中的边缘。
* 我们可以通过数据来学习卷积核。